In [1]:
import pandas as pd
data_path = "../../data/open_source_8454_combine_short_description.csv"
df = pd.read_csv(data_path)
# df_label_0 = df[df["Label"] == 0]
# df_label_0_to_drop = df_label_0.sample(n=3000, random_state=42)
# df = df.drop(df_label_0_to_drop.index)
df

,Inbound Message,Label
0,login issue verified user details employee man...,0
1,outlook hello team meetings skype meetings etc...,0
2,cant log vpn cannot log vpn best,0
3,unable access tool page unable access tool page,0
4,skype error skype error,0
...,...,...
8449,emails coming mail good afternoon receiving em...,22
8450,telephony software issue telephony software issue,0
8451,vip windows password reset tifpdchb pedxruyf v...,0
8452,machine est funcionando unable access machine ...,44


In [2]:
import sys
import os
sys.path.append(os.path.abspath('../ml_lib'))

from config import *
from dataset import CustomDataset
from train_eval import train_epoch, eval_model
from device import get_device_info

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import StratifiedKFold

device = get_device_info()

messages, labels = df['Inbound Message'].tolist(), df['Label'].tolist()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# KFold 初始化
skf = StratifiedKFold(n_splits=KFOLD_SPLIT, shuffle=True, random_state=SEED)

Using device: cuda
CUDA Available: True
CUDA Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Current GPU Memory Allocated: 0.00 GB
Current GPU Memory Cached: 0.00 GB


In [4]:
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
import numpy as np
import time

all_fold_results = []

start_time = time.time()

for fold, (train_idx, val_idx) in enumerate(skf.split(messages, labels)):
    print(f"Fold {fold + 1}")
    
    train_messages = [messages[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_messages = [messages[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]

    print("Training set label distribution:", {label: train_labels.count(label) for label in set(train_labels)})
    print("Validation set label distribution:", {label: val_labels.count(label) for label in set(val_labels)})
    
    train_dataset = CustomDataset(train_messages, train_labels, tokenizer, MAX_LENGTH)
    val_dataset = CustomDataset(val_messages, val_labels, tokenizer, MAX_LENGTH)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    NUM_LABELS = len(set(train_labels) | set(val_labels))
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=NUM_LABELS)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    criterion = CrossEntropyLoss()

    epoch_results = []
    best_macro_f1 = 0
    epochs_without_improvement = 0
    
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        train_loss = train_epoch(model, train_dataloader, optimizer, criterion, device)
        metrics = eval_model(model, val_dataloader, criterion, device, num_labels=NUM_LABELS)
        epoch_results.append(metrics)
        print(metrics)

        current_macro_f1 = metrics["macro_f1-score"]
        
        if current_macro_f1 > best_macro_f1:
            best_macro_f1 = current_macro_f1  # 更新最佳 macro_f1-score
            epochs_without_improvement = 0   # 重置計數
        else:
            epochs_without_improvement += 1  # 增加計數
        
        # 若連續 patience 3 次沒有提升，則停止訓練
        if epochs_without_improvement >= PATIENCE:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break
    
    # avg_metrics = {metric: np.mean([epoch[metric] for epoch in epoch_results]) for metric in epoch_results[0].keys()}
    # all_fold_results.append(avg_metrics)
    best_epoch_metrics = max(epoch_results, key=lambda x: x["macro_f1-score"])
    all_fold_results.append(best_epoch_metrics)

end_time = time.time()
print(f"Total time: {end_time - start_time:.2f}s")

final_metrics = {metric: np.mean([result[metric] for result in all_fold_results]) for metric in all_fold_results[0].keys()}
print("Final 10-fold Cross-Validation Results:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value}")

D:\Anaconda3\envs\cuda\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


Fold 1
Training set label distribution: {0: 3576, 1: 28, 2: 126, 3: 27, 4: 231, 5: 130, 6: 107, 7: 35, 8: 76, 9: 73, 10: 79, 11: 194, 12: 217, 13: 32, 14: 26, 15: 28, 16: 23, 17: 260, 18: 104, 19: 51, 20: 16, 21: 39, 22: 88, 23: 180, 24: 18, 25: 56, 26: 97, 27: 55, 28: 13, 29: 15, 30: 17, 31: 90, 32: 40, 33: 36, 34: 34, 35: 13, 36: 32, 37: 24, 38: 7, 39: 116, 40: 13, 41: 9, 42: 166, 43: 18, 44: 22, 45: 10, 46: 61, 47: 595, 48: 227, 49: 78}
Validation set label distribution: {0: 398, 1: 3, 2: 14, 3: 3, 4: 26, 5: 15, 6: 11, 7: 4, 8: 9, 9: 8, 10: 9, 11: 21, 12: 24, 13: 4, 14: 3, 15: 3, 16: 2, 17: 29, 18: 12, 19: 5, 20: 2, 21: 5, 22: 9, 23: 20, 24: 2, 25: 7, 26: 10, 27: 6, 28: 2, 29: 1, 30: 2, 31: 10, 32: 5, 33: 4, 34: 3, 35: 2, 36: 3, 37: 3, 39: 13, 40: 1, 41: 2, 42: 18, 43: 2, 44: 3, 45: 1, 46: 7, 47: 66, 48: 25, 49: 9}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20


100%|██████████| 476/476 [03:20<00:00,  2.37it/s]


...

{'val_loss': 1.600139562251433, 'val_accuracy': 0.6804733727810651, 'precision': 0.6686431805422406, 'recall': 0.6804733727810651, 'f1-score': 0.6588820861304819, 'macro_f1-score': 0.3590705913560073, 'balanced_accuracy': 0.35184817091330045, 'mcc': 0.5742432507298693}
Epoch 11/20


100%|██████████| 476/476 [03:22<00:00,  2.35it/s]


{'val_loss': 1.6277132827155996, 'val_accuracy': 0.676923076923077, 'precision': 0.6935773592662914, 'recall': 0.676923076923077, 'f1-score': 0.6684453454132725, 'macro_f1-score': 0.3775333153075631, 'balanced_accuracy': 0.3838581630274517, 'mcc': 0.5794866431710132}
Epoch 12/20


100%|██████████| 476/476 [03:22<00:00,  2.35it/s]


{'val_loss': 1.6830121762347672, 'val_accuracy': 0.6757396449704142, 'precision': 0.6730197021246516, 'recall': 0.6757396449704142, 'f1-score': 0.6608908448346359, 'macro_f1-score': 0.3623168478168528, 'balanced_accuracy': 0.3586278927465466, 'mcc': 0.5722362623960897}
Early stopping triggered at epoch 12
Total time: 27978.59s
Final 10-fold Cross-Validation Results:
val_loss: 1.551667298770176
val_accuracy: 0.6857130667114301
precision: 0.691283319762055
recall: 0.6857130667114301
f1-score: 0.6745159165180878
macro_f1-score: 0.4090068139189535
balanced_accuracy: 0.4089002592791909
mcc: 0.5850479364305337


In [5]:
epoch_results

[{'val_loss': 1.4919480012272888,
  'val_accuracy': 0.6236686390532544,
  'precision': 0.4896644390915901,
  'recall': 0.6236686390532544,
  'f1-score': 0.5304494480560625,
  'macro_f1-score': 0.14012108924683433,
  'balanced_accuracy': 0.15261628317938222,
  'mcc': 0.46638810948174875},
 {'val_loss': 1.285447598628278,
  'val_accuracy': 0.6556213017751479,
  'precision': 0.6197426937752447,
  'recall': 0.6556213017751479,
  'f1-score': 0.624215929934027,
  'macro_f1-score': 0.24528994493572645,
  'balanced_accuracy': 0.2624976487223424,
  'mcc': 0.5454293425256885},
 {'val_loss': 1.2404891831694909,
  'val_accuracy': 0.6733727810650888,
  'precision': 0.6224066948174566,
  'recall': 0.6733727810650888,
  'f1-score': 0.6319175164329187,
  'macro_f1-score': 0.26120893022866293,
  'balanced_accuracy': 0.2651144657986879,
  'mcc': 0.5539382081372957},
 {'val_loss': 1.2563837676678065,
  'val_accuracy': 0.6970414201183432,
  'precision': 0.6579994247354594,
  'recall': 0.6970414201183432,
